## Auto-Filler 

In [1]:
import string
import pandas as pd
from hazm import sent_tokenize, word_tokenize, Normalizer
import nltk
from nltk import bigrams, trigrams
from collections import Counter

In [2]:
comments_df = pd.read_csv('Datasets\digikala_comment.csv')
comments_df.head()

,comment
0,نسبت به قیمتش ارزش خرید داره\nجاداره، طراحیش ق...
1,چند ماهی میشه که گرفتمش‌. برای برنامه نویسی و ...
2,پراید ستون جدید
3,اقا همه چیش خوبه فقط از پایین زیاد حاشیه داره ...
4,گوسی هو اوی p10 lite سیپیو و دوربین و رمش از ا...


### Pre-Processing Function

In [3]:
def preprocess_sentenes(sentences_DataFrame):
    # Convert comments into sentences
    normalizer = Normalizer()
    comments_df['sentences'] = comments_df['comment'].apply(lambda comment: sent_tokenize(comment))
    comments_df.drop('comment', axis=1, inplace=True)

    # Convert DataFrame into a list of sentences
    sentences_list = comments_df['sentences'].tolist()
    flat_sentences_list = [sentence for sublist in sentences_list for sentence in sublist]

    normalized_sentences = [normalizer.normalize(s) for s in flat_sentences_list]

    # Remove Zero-Width Non-Joiners
    sentences = [sentence.replace('\u200c', ' ') for sentence in normalized_sentences]

    # Remove punctuations from sentences
    punctuations = string.punctuation + '،' + '؟'
    cleaned_sentences = []
    for sentence in sentences:
        cleaned_sentence = ''.join(char for char in sentence if char not in punctuations)
        cleaned_sentences.append(cleaned_sentence)

    return cleaned_sentences

In [4]:
preprocessed_sentenes = preprocess_sentenes(comments_df)

In [5]:
for i in range(10):
    print(preprocessed_sentenes[i])

نسبت به قیمتش ارزش خرید داره جاداره طراحیش قشنگه تنها مشکلش بندهای ضعیفش هست که باعث میشه استحکام چندانی نداشنه باشه
چند ماهی میشه که گرفتمش
برای برنامه نویسی و کارای گرافیکی ازش استفاده می کنم
واقعا از هر لحاظ بگین عالیه
پراید ستون جدید
اقا همه چیش خوبه فقط از پایین زیاد حاشیه داره که با روشن شدن گوشی بیشتر هم میشه
و نکته دیگه اینکه به خاطر این که اطرافش یه کوچلو خمیده هست گلس بعد یه مدتی جدا مشیه
ولی در کل با این قیمت بهترین گوشی هست و همه چی داره از دوربین گرفته تا رم و سی پی یو و گرافیک و حسگر های مختلف و خیلی چیزای دیگه
گوسی هو اوی p ۱۰ lite سیپیو و دوربین و رمش از این خیلی بهتره خودتون میتونین برین تمام مقایسه های p ۱۰ liteو این گوشیو ببینین
چادر سبک و زیباییه دوختشم عالیه


### Language Modeling

In [6]:
def extract_ngrams(sentences):
    # Tokenize sentences into words
    tokenized_sentences = [word_tokenize(sentence) for sentence in sentences]
    
    unigrams = [word for sentence in tokenized_sentences for word in sentence]
    bigrams_list = list(bigrams(unigrams))
    trigrams_list = list(trigrams(unigrams))
    
    return unigrams, bigrams_list, trigrams_list

def count_ngrams(unigrams, bigrams_list, trigrams_list):
    unigram_counts = Counter(unigrams)
    bigram_counts = Counter(bigrams_list)
    trigram_counts = Counter(trigrams_list)
    
    return unigram_counts, bigram_counts, trigram_counts

def report_most_frequent_ngrams(unigram_counts, bigram_counts, trigram_counts):
    print("Most frequent unigrams:")
    for word, count in unigram_counts.most_common(8):
        print(f"{word}: {count}")

    print("\nMost frequent bigrams:")
    for bigram, count in bigram_counts.most_common(8):
        print(f"{' '.join(bigram)}: {count}")

    print("\nMost frequent trigrams:")
    for trigram, count in trigram_counts.most_common(8):
        print(f"{' '.join(trigram)}: {count}")

In [7]:
unigrams, bigrams, trigrams = extract_ngrams(preprocessed_sentenes)
uc, bc, tc = count_ngrams(unigrams, bigrams, trigrams)
report_most_frequent_ngrams(uc, bc, tc)

Most frequent unigrams:
و: 370
از: 215
که: 187
این: 164
به: 156
هم: 123
خیلی: 112
رو: 108

Most frequent bigrams:
می کنم: 28
نسبت به: 23
از این: 19
این گوشی: 18
در کل: 17
پیشنهاد می: 15
دیجی کالا: 15
بعد از: 14

Most frequent trigrams:
پیشنهاد می کنم: 15
نسبت به قیمتش: 9
ممنون از دیجی: 9
حتما پیشنهاد می: 8
از دیجی کالا: 8
به نظر من: 7
این گوشی رو: 6
می کنم ممنون: 6
